In [ ]:
from OperacionesCSV import importar_CSV
from Neo4HungerGames import Neo4HungerGames
from Neo4Dataframes import Neo4Dataframes
from Neo4Graphics import Neo4Graphics
import pandas as pd
from flask import Flask, render_template, request, Response
from Controller import Controller

import base64
def encode_image(response):
    return base64.b64encode(response.get_data()).decode("utf-8")

In [ ]:
uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"
neoConnection = Neo4HungerGames(uri, user, password)

In [ ]:
app = Flask(__name__)
graphs = Neo4Graphics(neoConnection)
# Ruta principal
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/characters', methods=['GET'])
def characters():
    districts = neoConnection.get_all_districts()
    return render_template('characters.html', districts=districts)

@app.route('/characters/all', methods=['GET'])
def all_characters():
    characters = neoConnection.get_all_characters()
    return render_template('characters_list.html', characters=characters)

@app.route('/characters/create', methods=['POST'])
def create_character():
    return Controller.create_character(neoConnection, request=request)

@app.route("/characters/link_game", methods=["GET", "POST"])
def link_character_game():
    return Controller.link_character_game(neoConnection, request=request)

@app.route("/characters/update/<int:character_id>", methods=["GET"])
def edit_character_form(character_id):
    character = neoConnection.get_character_by_id(character_id)
    return render_template("update_character.html", character=character, character_id=character_id)

@app.route("/characters/update/<int:character_id>", methods=["POST"])
def update_character(character_id):
    return Controller.update_character(neoConnection, request, character_id)

@app.route("/characters/delete", methods=["POST"])
def delete_character():
    return Controller.delete_character(neoConnection, request)

@app.route("/characters/delete/<character_id>", methods=["POST"])
def delete_character_by_id(character_id):
    return Controller.delete_character_by_id(neoConnection, character_id)

@app.route("/characters/link_family", methods=["GET"])
def show_family_link_form():
    characters = neoConnection.get_all_characters_id_and_name()
    return render_template("link_family.html", characters=characters)

@app.route("/characters/link_family", methods=["POST"])
def create_family_link():
    return Controller.create_family_link(neoConnection, request)

@app.route("/characters/family/all")
def get_all_family_links():
    return Controller.get_all_family_links(neoConnection)

@app.route("/characters/family/delete", methods=["POST"])
def delete_family_link():
    return Controller.delete_family_link(neoConnection, request.form)

@app.route("/links", methods=["GET"])
def list_links():
    return Controller.list_links(neoConnection)

@app.route("/links/delete", methods=["POST"])
def delete_link():
    return Controller.delete_link(neoConnection, request.form)


@app.route("/histograms", methods=["GET"])
def histograms():
    books = encode_image(graphs.characters_per_book())
    districts = encode_image(graphs.characters_per_district())
    alliances = encode_image(graphs.characters_per_alliance())
    games = encode_image(graphs.characters_per_game())
    kills = encode_image(graphs.kills_per_cause())

    return render_template("histograms.html",
                           books=books,
                           districts=districts,
                           alliances=alliances,
                           games=games,
                           kills=kills)

@app.route("/characters/other_books", methods=["GET", "POST"])
def characters_other_books():
    books = neoConnection.get_all_books()  # Para popular el select en HTML
    if request.method == "POST":
        title = request.form.get("title")
        characters = neoConnection.get_characters_from_other_books(title)
        return render_template("characters_other_books.html", books=books, characters=characters, selected_title=title)
    return render_template("characters_other_books.html", books=books, characters=None, selected_title=None)

@app.route("/books", methods=["GET", "POST"])
def book_characters():
    books = neoConnection.get_all_books()  # Para popular el select en HTML
    if request.method == "POST":
        title = request.form.get("title")
        characters = neoConnection.get_characters_from_a_book(title)
        return render_template("book_characters.html", books=books, characters=characters, selected_title=title)
    return render_template("book_characters.html", books=books, characters=None, selected_title=None)


if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=8080,use_reloader=False)

In [ ]:
neoConnection.close()

In [ ]:
ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

hgDF = importar_CSV(ruta_csv)
hgDF['ID'] = hgDF.index
dataConnection = Neo4Dataframes(uri, user, password)
dataConnection.delete_all_links_and_nodes()
dataConnection.create_nodes_links_from_df(hgDF)
dataConnection.close()